In [3]:
(version)

"8.3"

In [4]:
(require racket
         racket/list
         racket/format
         (prefix-in pict: pict))

In [5]:
(require bhdl)

#<procedure:parameter-procedure>


In [6]:
(define (add-ghost-top p sep)
  "Add some space on top of picture"
  (vc-append (pict:ghost (pict:rectangle 1 sep)) p))

In [12]:
(define spreadboard
  (make-circuit
    (part [jumper-mini-nano-SCL (PinHeader 3)]  ;j3
          [jumper-mini-nano-SDA (PinHeader 3)]  ;j4
          [jumper-micro-SDA (PinHeader 3)]      ;j10
          [jumper-micro-SCL (PinHeader 3)]      ;j11
          [jumper-micro-mini-VCC (PinHeader 3)] ;J1
          [jumper-mkr-MOSI (PinHeader 3)]       ;J5
          [jumper-mkr-SCK (PinHeader 3)]        ;J6
          [jumper-mkr-SDA (PinHeader 3)]        ;J8
          [jumper-mkr-SCL (PinHeader 3)]        ;J9
          [uno1 (Arduino-Uno)]
          [uno2 (Arduino-Uno)]
          [uno3 (Arduino-Uno-ICSP)]
          [uno4 (Arduino-Uno-ICSP)]
          [micro (Arduino-Micro)]
          [mini (Arduino-Mini)]
          [nano (Arduino-Nano)]
          [mkr (Arduino-MKR)])
    ;; Connect GPIO pins of different Arduinos
    (wire
      (bus
        (uno1  [A0 A1 A2 A3 A4 A5 -  -  -  -  D2 D3 D4 D5 D6 D7 D8 D9 D10 D11 D12 D13 -   GND 3V3 5V])
        (uno2  [A0 A1 A2 A3 A4 A5 -  -  -  -  D2 D3 D4 D5 D6 D7 D8 D9 D10 D11 D12 D13 -   GND 3V3 5V])
        (uno3  [A0 A1 A2 A3 A4 A5 -  -  -  -  D2 D3 D4 D5 D6 D7 D8 D9 D10 D11 D12 D13 VCC GND 3V3 5V])
        (uno4  [A0 A1 A2 A3 A4 A5 -  -  -  -  D2 D3 D4 D5 D6 D7 D8 D9 D10 D11 D12 D13 VCC GND 3V3 5V])
        (micro [A0 A1 A2 A3 -  -  -  -  -  -  -  -  D4 D5 D6 D7 D8 D9 D10 -   -   -   -   GND -   -])
        (mini  [A0 A1 A2 A3 -  -  A6 A7 -  -  D2 D3 D4 D5 D6 D7 D8 D9 D10 D11 D12 D13 -   GND -   -])
        (nano  [A0 A1 A2 A3 -  -  A6 A7 D0 D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 D11 D12 D13 -   GND 3V3 5V])
        (mkr   [A0 A1 A2 A3 A4 A5 A6 -  D0 D1 D2 D3 D4 D5 D6 D7 -  -  D10 -   -   D13 -   GND 3V3 5V]))
      
      ;; TODO define some global signal for e.g. uno1.SDA
     
      (list (bus (jumper-mkr-SDA [1 2 3])
                 ([uno1.D11 mkr.D11 uno1.SDA]))
            (bus (jumper-mkr-SCL [1 2 3])
                 ([uno1.D12 mkr.D12 uno1.SCL]))
            (bus (jumper-mkr-MOSI [1 2 3])
                 ([uno3.D8 mkr.D8 uno3.MOSI]))
            (bus (jumper-mkr-SCK [1 2 3])
                 ([uno3.D9 mkr.D9 uno3.SCK]))
            (bus (jumper-micro-SDA [1 2 3])
                 ([uno1.D2 micro.D2 uno1.SDA]))
            (bus (jumper-micro-SCL [1 2 3])
                 ([uno1.D3 micro.D3 uno1.SCL]))
            ;; jumpter mini/nano
            (net jumper-mini-nano-SCL.2 mini.A5 nano.A5)
            (net jumper-mini-nano-SCL.1 uno1.A5)
            (net jumper-mini-nano-SCL.3 uno1.SCL)
            (net jumper-mini-nano-SDA.2 mini.A4 nano.A4)
            (net jumper-mini-nano-SDA.1 uno1.A4)
            (net jumper-mini-nano-SDA.3 uno1.SDA)
            ;; jumper micro mini
            (net jumper-micro-mini-VCC.2 micro.VCC mini.VCC)
            (net jumper-micro-mini-VCC.1 uno1.3V3)
            (net jumper-micro-mini-VCC.3 uno1.5V)
            ))
    (layout (let ([g1 (ct-superimpose
                        uno1
                        (vc-append
                          10
                          (add-ghost-top mini 50)
                          (vc-append (rotate jumper-mini-nano-SCL (/ pi 2))
                                     (rotate jumper-mini-nano-SDA (/ pi 2)))))]
                  [g2 (ct-superimpose
                        uno2
                        (vc-append
                          10
                          (add-ghost-top micro 50)
                          (vc-append (rotate jumper-micro-SDA (/ pi 2))
                                     (rotate jumper-micro-SCL (/ pi 2))))
                        (rotate jumper-micro-mini-VCC (/ pi 2)))]
                  [g3 (ct-superimpose
                        uno3
                        (hc-append 10
                                   (vc-append (rotate jumper-mkr-MOSI (/ pi 2))
                                              (rotate jumper-mkr-SCK (/ pi 2))
                                              (rotate jumper-mkr-SDA (/ pi 2))
                                              (rotate jumper-mkr-SCL (/ pi 2)))
                                   (add-ghost-top (rotate mkr pi) 50)))]
                  [g4 (ct-superimpose uno4 (add-ghost-top nano 50))])
              (inset (vc-append 
                       (ht-append 20 g1 g2)
                       (ht-append 20 g3 g4))
                     20 20 20 -50)))))

In [13]:
(show-layout spreadboard)

(pict '(picture 516.8992649059583 634.0539066545798 (put 20 -50 #2=(picture 476.89926490595826 664.0539066545798 (put 0.0 342.0269533272899 #3=(picture 476.89926490595826 322.0269533272899 (put 0 0.0 #4=(picture 228.44963245297913 322.0269533272899 (put 0.0 0.0 #0=(prog #<procedure:...t/private/utils.rkt:1287:8> 322.0269533272899)) (put 76.14987748432637 49.76890650953919 #5=(picture 76.14987748432638 272.2580468177507 (put 0.0 60.83684577543026 #6=(picture 76.14987748432638 211.42120104232043 (put 37.57493874216319 161.42120104232043 #7=(picture 1 50)) (put 0.0 0 #8=(prog #<procedure:...t/private/utils.rkt:1287:8> 141.42120104232043)))) (put 19.5514168006721 0 #10=(picture 37.04704388298218 50.83684577543026 (put 0.0 35.41842288771513 #11=(prog #<procedure:...t/private/utils.rkt:1379:20> 15.41842288771513)) (put 0.0 0 #12=(prog #<procedure:...t/private/utils.rkt:1379:20> 15.41842288771513)))))))) (put 248.44963245297913 0.0 #18=(picture 228.44963245297913 322.0269533272899 (put 0.0 0.0 #0#) (put 76.14987748432637 44.32962954637304 #20=(picture 76.14987748432638 277.69732378091686 (put 0.0 60.83684577543026 #21=(picture 76.14987748432638 216.8604780054866 (put 37.57493874216319 166.8604780054866 #22=(picture 1 50)) (put 0.0 0 #23=(prog #<procedure:...t/private/utils.rkt:1287:8> 146.8604780054866)))) (put 19.5514168006721 0 #25=(picture 37.04704388298218 50.83684577543026 (put 0.0 35.41842288771513 #26=(prog #<procedure:...t/private/utils.rkt:1379:20> 15.41842288771513)) (put 0.0 0 #27=(prog #<procedure:...t/private/utils.rkt:1379:20> 15.41842288771513)))))) (put 95.70129428499848 306.6085304395748 #19=(prog #<procedure:...t/private/utils.rkt:1379:20> 15.41842288771513)))))) (put 0.0 0 #33=(picture 476.89926490595826 322.0269533272899 (put 0 0.0 #34=(picture 228.44963245297913 322.0269533272899 (put 0.0 0.0 #1=(prog #<procedure:...t/private/utils.rkt:1287:8> 322.0269533272899)) (put 36.308524653336775 1.8202130216460546 #35=(picture 155.83258314630558 320.20674030564385 (put 0 99.26652437739168 #36=(picture 37.04704388298218 121.67369155086051 (put 0.0 106.25526866314539 #37=(prog #<procedure:...t/private/utils.rkt:1379:20> 15.41842288771513)) (put 0.0 0 #38=(picture 37.04704388298218 86.25526866314539 (put 0.0 70.83684577543026 #39=(prog #<procedure:...t/private/utils.rkt:1379:20> 15.41842288771513)) (put 0.0 0 #40=(picture 37.04704388298218 50.83684577543026 (put 0.0 35.41842288771513 #41=(prog #<procedure:...t/private/utils.rkt:1379:20> 15.41842288771513)) (put 0.0 0 #42=(prog #<procedure:...t/private/utils.rkt:1379:20> 15.41842288771513)))))))) (put 47.04704388298218 0.0 #47=(picture 108.78553926332341 320.20674030564385 (put 53.89276963166171 270.20674030564385 #48=(picture 1 50)) (put 0.0 0 #49=(prog #<procedure:...t/private/utils.rkt:1379:20> 250.20674030564382)))))))) (put 248.44963245297913 0.0 #53=(picture 228.44963245297913 322.0269533272899 (put 0.0 0.0 #1#) (put 75.06202209169314 58.38869343857425 #54=(picture 78.32558826959284 263.63825988871565 (put 38.66279413479642 213.63825988871562 #55=(picture 1 50)) (put 0.0 0 #56=(prog #<procedure:...t/private/utils.rkt:1287:8> 193.63825988871562))))))))))) 516.8992649059583 634.0539066545798 27.709211443857562 16.00368118700689 (list (child (pict #2# 476.89926490595826 664.0539066545798 7.709211443857564 66.00368118700689 (list (child (pict #3# 476.89926490595826 322.0269533272899 7.709211443857564 52.03884099023061 (list (child (pict #4# 228.44963245297913 322.0269533272899 50.0 57.47811795339676 (list (child (pict #5# 76.14987748432638 272.2580468177507 50 7.709211443857566 (list (child (pict #6# 76.14987748432638 211.42120104232043 50 70.71060052116022 (list (child (pict #7# 1 50 50 0 (list (child (pict '(prog #<procedure:...t/private/utils.rkt:382:8> 50) 1 50 50 0 '() #f #f) 0 0 1 1 0 0)) #f #f) 37.57493874216319 161.42120104232043 1 1 0 0) (child #9=(pict #8# 76.14987748432638 141.42120104232043 70.71060052116022 70.71060052116022 '() (bbox 0.0 0.0 76.14987748432638 

In [10]:
(parameterize ([current-directory "./out/spreadboard"]
               [padding-general 2])
    (circuit-export spreadboard
                    #:auto-place #f
                    #:formats '(kicad pdf png)))

generating KiCAD PCB ..
"DEBUG: /var/folders/jd/pqbc6_7j0y5gc2kjxjntw8c40000gn/T/PKGS-src-ROOT-BHDL-syst----iVc-fpio-main.rkt-95-15_16388432681638843268711"
"DEBUG: /var/folders/jd/pqbc6_7j0y5gc2kjxjntw8c40000gn/T/PKGS-src-ROOT-BHDL-syst----iVc-fpio-main.rkt-95-15_16388432681638843268718"
"DEBUG: /var/folders/jd/pqbc6_7j0y5gc2kjxjntw8c40000gn/T/PKGS-src-ROOT-BHDL-syst----iVc-fpio-main.rkt-95-15_16388432681638843268731"
"DEBUG: /var/folders/jd/pqbc6_7j0y5gc2kjxjntw8c40000gn/T/PKGS-src-ROOT-BHDL-syst----iVc-fpio-main.rkt-95-15_16388432681638843268744"
"DEBUG: /var/folders/jd/pqbc6_7j0y5gc2kjxjntw8c40000gn/T/PKGS-src-ROOT-BHDL-syst----iVc-fpio-main.rkt-95-15_16388432681638843268749"
"DEBUG: /var/folders/jd/pqbc6_7j0y5gc2kjxjntw8c40000gn/T/PKGS-src-ROOT-BHDL-syst----iVc-fpio-main.rkt-95-15_16388432681638843268755"
"DEBUG: /var/folders/jd/pqbc6_7j0y5gc2kjxjntw8c40000gn/T/PKGS-src-ROOT-BHDL-syst----iVc-fpio-main.rkt-95-15_16388432681638843268762"
link: /Users/hebi/git/bhdl.org/notebooks/out/

(pict '(prog #<procedure:...t/private/utils.rkt:1287:8> 480.0) 391.31002041128716 480.0 480.0 0.0 (list (child (pict '(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #5=(picture 516.8992649059583 634.0539066545798 (put 0 0 #7=(picture 516.8992649059583 634.0539066545798 (put 0 0 #9=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #14=(picture 516.8992649059583 634.0539066545798 (put 0 0 #16=(picture 516.8992649059583 634.0539066545798 (put 0 0 #18=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #23=(picture 516.8992649059583 634.0539066545798 (put 0 0 #25=(picture 516.8992649059583 634.0539066545798 (put 0 0 #27=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #32=(picture 516.8992649059583 634.0539066545798 (put 0 0 #34=(picture 516.8992649059583 634.0539066545798 (put 0 0 #36=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #41=(picture 516.8992649059583 634.0539066545798 (put 0 0 #43=(picture 516.8992649059583 634.0539066545798 (put 0 0 #45=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #50=(picture 516.8992649059583 634.0539066545798 (put 0 0 #52=(picture 516.8992649059583 634.0539066545798 (put 0 0 #54=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #59=(picture 516.8992649059583 634.0539066545798 (put 0 0 #61=(picture 516.8992649059583 634.0539066545798 (put 0 0 #63=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #68=(picture 516.8992649059583 634.0539066545798 (put 0 0 #70=(picture 516.8992649059583 634.0539066545798 (put 0 0 #72=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #77=(picture 516.8992649059583 634.0539066545798 (put 0 0 #79=(picture 516.8992649059583 634.0539066545798 (put 0 0 #81=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #86=(picture 516.8992649059583 634.0539066545798 (put 0 0 #88=(picture 516.8992649059583 634.0539066545798 (put 0 0 #90=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #95=(picture 516.8992649059583 634.0539066545798 (put 0 0 #97=(picture 516.8992649059583 634.0539066545798 (put 0 0 #99=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #104=(picture 516.8992649059583 634.0539066545798 (put 0 0 #106=(picture 516.8992649059583 634.0539066545798 (put 0 0 #108=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #113=(picture 516.8992649059583 634.0539066545798 (put 0 0 #115=(picture 516.8992649059583 634.0539066545798 (put 0 0 #117=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #122=(picture 516.8992649059583 634.0539066545798 (put 0 0 #124=(picture 516.8992649059583 634.0539066545798 (put 0 0 #126=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #131=(picture 516.8992649059583 634.0539066545798 (put 0 0 #133=(picture 516.8992649059583 634.0539066545798 (put 0 0 #135=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #140=(picture 516.8992649059583 634.0539066545798 (put 0 0 #142=(picture 516.8992649059583 634.0539066545798 (put 0 0 #144=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #149=(picture 516.8992649059583 634.0539066545798 (put 0 0 #151=(picture 516.8992649059583 634.0539066545798 (put 0 0 #153=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #158=(picture 516.8992649059583 634.0539066545798 (put 0 0 #160=(picture 516.8992649059583 634.0539066545798 (put 0 0 #162=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #167=(picture 516.8992649059583 634.0539066545798 (put 0 0 #169=(picture 516.8992649059583 634.0539066545798 (put 0 0 #171=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #176=(picture 516.8992649059583 634.0539066545798 (put 0 0 #178=(picture 516.8992649059583 634.0539066545798 (put 0 0 #180=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #185=(picture 516.8992649059583 634.0539066545798 (put 0 0 #187=(picture 516.8992649059583 634.0539066545798 (put 0 0 #189=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #194=(picture 516.8992649059583 634.0539066545798 (put 0 0 